# 10K Dataset

In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper

from tqdm import tqdm

## Download NLP Corpora
Use the stopwords and wordnet from nltk to preprocess the dataset later

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/kathy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kathy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Get 10Ks Dataset
We will use the company's unique CIK (Central Index Key) to identify each company.  
Find the CIK through: https://www.sec.gov/edgar/searchedgar/companysearch.html

In [3]:
# Set the company CIK here
cik_lookup = {
    'AMZN': '0001018724',
    'FB': '0001326801',   
    'GOOG': '0001652044',
    'MSFT': '0000789019',
    'TSLA': '0001318605'}

additional_cik = {
    'AEP': '0000004904',
    'AXP': '0000004962',
    'BA': '0000012927', 
    'BK': '0001390777',
    'CAT': '0000018230',
    'DE': '0000315189',
    'DIS': '0001001039',
    'DTE': '0000936340',
    'ED': '0001047862',
    'EMR': '0000032604',
    'ETN': '0001551182',
    'GE': '0000040545',
    'IBM': '0000051143',
    'IP': '0000051434',
    'JNJ': '0000200406',
    'KO': '0000021344',
    'LLY': '0000059478',
    'MCD': '0000063908',
    'MO': '0000764180',
    'MRK': '0000310158',
    'MRO': '0000101778',
    'PCG': '0001004980',
    'PEP': '0000077476',
    'PFE': '0000078003',
    'PG': '0000080424',
    'PNR': '0000077360',
    'SYY': '0000096021',
    'TXN': '0000097476',
    'UTX': '0000101829',
    'WFC': '0000072971',
    'WMT': '0000104169',
    'WY': '0000106535',
    'XOM': '0000034088'}

In [4]:
# Use SecAPI to get the 10Ks data
sec_api = project_helper.SecAPI()

In [5]:
# Pull a lost of filled 10-ks from the API for each company
from bs4 import BeautifulSoup

def get_sec_data(cik, doc_type, start=0, count=60):
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
        .format(cik, doc_type, start, count)
    sec_data = sec_api.get(rss_url)
    feed = BeautifulSoup(sec_data.encode('ascii'), 'xml').feed
    entries = [
        (
            entry.content.find('filing-href').getText(),
            entry.content.find('filing-type').getText(),
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)]

    return entries

In [6]:
# Pull the data, and show one of the examples
example_ticker = 'AMZN'
sec_data = {}

for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik, '10-K')

pprint.pprint(sec_data[example_ticker][:5])

[('https://www.sec.gov/Archives/edgar/data/1018724/000101872420000004/0001018724-20-000004-index.htm',
  '10-K',
  '2020-01-31'),
 ('https://www.sec.gov/Archives/edgar/data/1018724/000101872419000004/0001018724-19-000004-index.htm',
  '10-K',
  '2019-02-01'),
 ('https://www.sec.gov/Archives/edgar/data/1018724/000101872418000005/0001018724-18-000005-index.htm',
  '10-K',
  '2018-02-02'),
 ('https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/0001018724-17-000011-index.htm',
  '10-K',
  '2017-02-10'),
 ('https://www.sec.gov/Archives/edgar/data/1018724/000101872416000172/0001018724-16-000172-index.htm',
  '10-K',
  '2016-01-29')]


In [7]:
# Download fillings from the urls we get in last step
raw_fillings_by_ticker = {}

for ticker, data in sec_data.items():
    raw_fillings_by_ticker[ticker] = {}
    for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
        if (file_type == '10-K'):
            file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
            raw_fillings_by_ticker[ticker][file_date] = sec_api.get(file_url)


print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

Example Document:

<SEC-DOCUMENT>0001018724-20-000004.txt : 20200131
<SEC-HEADER>0001018724-20-000004.hdr.sgml : 20200131
<ACCEPTANCE-DATETIME>20200130204613
ACCESSION NUMBER:		0001018724-20-000004
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		109
CONFORMED PERIOD OF REPORT:	20191231
FILED AS OF DATE:		20200131
DATE AS OF CHANGE:		20200130

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			AMAZON COM INC
		CENTRAL INDEX KEY:			0001018724
		STANDARD INDUSTRIAL CLASSIFICATION:	RETAIL-CATALOG & MAIL-ORDER HOUSES [5961]
		IRS NUMBER:				911646860
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VALUES:
		FORM TYPE:		10-K
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	000-22513
		FILM NUMBER:		20562951

	BUSINESS ADDRESS:	
		STREET 1:		410 TERRY AVENUE NORTH
		CITY:			SEATTLE
		STATE:			WA
		ZIP:			98109
		BUSINESS PHONE:		2062661000

	MAIL ADDRESS:	
		STREET 1:		410 TERRY AVENUE NORTH
		CITY:			SEATTLE
		STATE:			WA
		ZIP:			98109
</SEC-HEADER>
<DOCUMENT>
<TYPE>10-K
<

In [8]:
# Get documents from the fillings
import re

# To return a list of documents from a filling
def get_documents(text):
    extracted_docs = []
    
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')   
    
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(text)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(text)]
    
    for doc_start_i, doc_end_i in zip(doc_start_is, doc_end_is):
            extracted_docs.append(text[doc_start_i:doc_end_i])
    
    return extracted_docs

In [9]:
# Extract the documents for the company
filling_documents_by_ticker = {}

for ticker, raw_fillings in raw_fillings_by_ticker.items():
    filling_documents_by_ticker[ticker] = {}
    for file_date, filling in tqdm(raw_fillings.items(), desc='Getting Documents from {} Fillings'.format(ticker), unit='filling'):
        filling_documents_by_ticker[ticker][file_date] = get_documents(filling)


print('\n\n'.join([
    'Document {} Filed on {}:\n{}...'.format(doc_i, file_date, doc[:200])
    for file_date, docs in filling_documents_by_ticker[example_ticker].items()
    for doc_i, doc in enumerate(docs)][:3]))

Getting Documents from TSLA Fillings: 100%|██████████| 10/10 [00:00<00:00, 32.21filling/s]

Document 0 Filed on 2020-01-31:

<TYPE>10-K
<SEQUENCE>1
<FILENAME>amzn-20191231x10k.htm
<DESCRIPTION>10-K
<TEXT>
<XBRL>
<?xml version="1.0" encoding="UTF-8"?>
<!--XBRL Document Created with Wdesk from Workiva-->
<!--p:c57a17684e854b...

Document 1 Filed on 2020-01-31:

<TYPE>EX-4.6
<SEQUENCE>2
<FILENAME>amzn-20191231xex46.htm
<DESCRIPTION>EXHIBIT 4.6
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>...

Document 2 Filed on 2020-01-31:

<TYPE>EX-21.1
<SEQUENCE>3
<FILENAME>amzn-20191231xex211.htm
<DESCRIPTION>EXHIBIT 21.1
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<ht...


In [10]:
# Define the function to get documents according to the type
def get_document_type(doc):
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    
    doc_type = type_pattern.findall(doc)[0][len('<TYPE>'):] 
    
    return doc_type.lower()

In [11]:
# Only get the 10-k documents for the companies we selected
ten_ks_by_ticker = {}

for ticker, filling_documents in filling_documents_by_ticker.items():
    ten_ks_by_ticker[ticker] = []
    for file_date, documents in filling_documents.items():
        for document in documents:
            if get_document_type(document) == '10-k':
                ten_ks_by_ticker[ticker].append({
                    'cik': cik_lookup[ticker],
                    'file': document,
                    'file_date': file_date})


project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['cik', 'file', 'file_date'])

[
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2019123...
    file_date: '2020-01-31'},
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2018123...
    file_date: '2019-02-01'},
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2017123...
    file_date: '2018-02-02'},
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2016123...
    file_date: '2017-02-10'},
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2015123...
    file_date: '2016-01-29'},
]


## Preprocess the Data

### Clean up - remove the html tags and lowercase all the text

In [12]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    return text

def clean_text(text):
    text = text.lower()
    text = remove_html_tags(text)
    
    return text

In [13]:
for ticker, ten_ks in ten_ks_by_ticker.items():
    for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
        ten_k['file_clean'] = clean_text(ten_k['file'])

project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_clean'])

Cleaning TSLA 10-Ks: 100%|██████████| 10/10 [00:18<00:00,  1.82s/10-K]

[
  {
    file_clean: '\n10-k\n1\namzn-20191231x10k.htm\n10-k\n\n\n\n\n\...},
  {
    file_clean: '\n10-k\n1\namzn-20181231x10k.htm\n10-k\n\n\n\n\n\...},
  {
    file_clean: '\n10-k\n1\namzn-20171231x10k.htm\n10-k\n\n\n\n\n\...},
  {
    file_clean: '\n10-k\n1\namzn-20161231x10k.htm\nform 10-k\n\n\n...},
  {
    file_clean: '\n10-k\n1\namzn-20151231x10k.htm\nform 10-k\n\n\n...},
]


### Lemmatize
Grouping together various inflections of a word to analyze them as a single item, identified by the word’s lemma

In [14]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


def lemmatize_words(words):
    lemmatized_words = [WordNetLemmatizer().lemmatize(word, 'v') for word in words]
    return lemmatized_words

In [15]:
word_pattern = re.compile('\w+')

for ticker, ten_ks in ten_ks_by_ticker.items():
    for ten_k in tqdm(ten_ks, desc='Lemmatize {} 10-Ks'.format(ticker), unit='10-K'):
        ten_k['file_lemma'] = lemmatize_words(word_pattern.findall(ten_k['file_clean']))


project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_lemma'])

Lemmatize TSLA 10-Ks: 100%|██████████| 10/10 [00:03<00:00,  3.1110-K/s]

[
  {
    file_lemma: '['10', 'k', '1', 'amzn', '20191231x10k', 'htm', '...},
  {
    file_lemma: '['10', 'k', '1', 'amzn', '20181231x10k', 'htm', '...},
  {
    file_lemma: '['10', 'k', '1', 'amzn', '20171231x10k', 'htm', '...},
  {
    file_lemma: '['10', 'k', '1', 'amzn', '20161231x10k', 'htm', '...},
  {
    file_lemma: '['10', 'k', '1', 'amzn', '20151231x10k', 'htm', '...},
]


### Remove Stopwords

In [16]:
from nltk.corpus import stopwords

lemma_english_stopwords = lemmatize_words(stopwords.words('english'))

for ticker, ten_ks in ten_ks_by_ticker.items():
    for ten_k in tqdm(ten_ks, desc='Remove Stop Words for {} 10-Ks'.format(ticker), unit='10-K'):
        ten_k['file_lemma'] = [word for word in ten_k['file_lemma'] if word not in lemma_english_stopwords]

Remove Stop Words for TSLA 10-Ks: 100%|██████████| 10/10 [00:01<00:00,  8.3510-K/s]


Here, the keys for each ten_k is ['cik', 'file', 'file_date', 'file_clean', 'file_lemma'].

### Transform the Data Format - from Dict to DataFrame

In [42]:
ten_ks_df_dict = {'date': [], 'company': [], 'ticker': [], 'lemma': [], 'doc': []}

for ticker, ten_ks in ten_ks_by_ticker.items():
    for ten_k in ten_ks:
        ten_ks_df_dict['date'].append(ten_k['file_date'])
        ten_ks_df_dict['company'].append(ticker)
        ten_ks_df_dict['ticker'].append(cik_lookup[ticker])
        ten_ks_df_dict['lemma'].append(ten_k['file_lemma'])
        ten_ks_df_dict['doc'].append(' '.join(ten_k['file_lemma']))

ten_ks_df = pd.DataFrame(ten_ks_df_dict)

ten_ks_df.head()

,date,company,ticker,text,doc
0,2020-01-31,AMZN,0001018724,"[10, k, 1, amzn, 20191231x10k, htm, 10, k, doc...",10 k 1 amzn 20191231x10k htm 10 k document 0 5...
1,2019-02-01,AMZN,0001018724,"[10, k, 1, amzn, 20181231x10k, htm, 10, k, doc...",10 k 1 amzn 20181231x10k htm 10 k document tab...
2,2018-02-02,AMZN,0001018724,"[10, k, 1, amzn, 20171231x10k, htm, 10, k, doc...",10 k 1 amzn 20171231x10k htm 10 k document uni...
3,2017-02-10,AMZN,0001018724,"[10, k, 1, amzn, 20161231x10k, htm, form, 10, ...",10 k 1 amzn 20161231x10k htm form 10 k documen...
4,2016-01-29,AMZN,0001018724,"[10, k, 1, amzn, 20151231x10k, htm, form, 10, ...",10 k 1 amzn 20151231x10k htm form 10 k 10 k un...


Since each company will only have one financial report each year, we will consider the 10ks text data is the same in the same year.

In [43]:
ten_ks_df['year'] = ten_ks_df['date'].apply(lambda x: x.split('-')[0])

ten_ks_df.head()

,date,company,ticker,text,doc,year
0,2020-01-31,AMZN,0001018724,"[10, k, 1, amzn, 20191231x10k, htm, 10, k, doc...",10 k 1 amzn 20191231x10k htm 10 k document 0 5...,2020
1,2019-02-01,AMZN,0001018724,"[10, k, 1, amzn, 20181231x10k, htm, 10, k, doc...",10 k 1 amzn 20181231x10k htm 10 k document tab...,2019
2,2018-02-02,AMZN,0001018724,"[10, k, 1, amzn, 20171231x10k, htm, 10, k, doc...",10 k 1 amzn 20171231x10k htm 10 k document uni...,2018
3,2017-02-10,AMZN,0001018724,"[10, k, 1, amzn, 20161231x10k, htm, form, 10, ...",10 k 1 amzn 20161231x10k htm form 10 k documen...,2017
4,2016-01-29,AMZN,0001018724,"[10, k, 1, amzn, 20151231x10k, htm, form, 10, ...",10 k 1 amzn 20151231x10k htm form 10 k 10 k un...,2016


## Use the Loughran-McDonald sentiment word lists

In [70]:
sentiments = ['negative', 'positive', 'uncertainty', 'litigious', 'constraining', 'interesting']

# Read the file
sentiment_df = pd.read_csv('LoughranMcDonald_MasterDictionary_2018.csv')
sentiment_df.columns = [column.lower() for column in sentiment_df.columns]

# Remove unused information
sentiment_df = sentiment_df[sentiments + ['word']]
sentiment_df[sentiments] = sentiment_df[sentiments].astype(bool)
sentiment_df = sentiment_df[(sentiment_df[sentiments]).any(1)]

# Apply the same preprocessing to these words as the 10-k words
sentiment_df['word'] = lemmatize_words(sentiment_df['word'].str.lower())
sentiment_df = sentiment_df.drop_duplicates('word')

sentiment_df.head()

,negative,positive,uncertainty,litigious,constraining,interesting,word
9,True,False,False,False,False,False,abandon
12,True,False,False,False,False,False,abandonment
13,True,False,False,False,False,False,abandonments
51,True,False,False,False,False,False,abdicate
54,True,False,False,False,False,False,abdication


In [72]:
# Use the positive words and negative words as vocabulary 
positive_voc = sentiment_df[sentiment_df['positive']]['word']
negative_voc = sentiment_df[sentiment_df['negative']]['word']

voc = positive_voc.append(negative_voc)

## Transform the 10ks documents to tf-idf vector

In [74]:
# Train the tfidf vectorizer based on the vocabulary
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(vocabulary=voc)
vec.fit(ten_ks_df['doc'].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=125                 able
336            abundance
338             abundant
438              acclaim
477           accomplish
481       accomplishment
482      accomplishments
620              achieve
623          achievement
624         achievements
906           adequately
1137         a...  wrongful
86041         wrongfully
86049            wrongly
Name: word, Length: 1764, dtype: object)

In [76]:
# Generate the tfidf sparse matrix for all the input 10k docs

tfidf = vec.transform(ten_ks_df['doc'])

In [77]:
print(tfidf.toarray())

[[0.08153187 0.         0.         ... 0.         0.         0.        ]
 [0.10263492 0.         0.         ... 0.         0.         0.        ]
 [0.09360538 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.24585583 0.         0.         ... 0.         0.         0.        ]
 [0.29710243 0.         0.         ... 0.         0.         0.        ]
 [0.27459558 0.         0.01493778 ... 0.         0.         0.        ]]


## TF-IDF

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Generate TFIDF values from documents for a certain sentiment
def get_tfidf(sentiment_words, docs):
    
    vec = TfidfVectorizer(vocabulary=sentiment_words)
    tfidf = vec.fit_transform(docs)
    
    return tfidf.toarray()
    

In [28]:
sentiment_tfidf_ten_ks = {}

for ticker, ten_ks in ten_ks_by_ticker.items():
    lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]
    if not lemma_docs:
        print('got')
        break
    
    sentiment_tfidf_ten_ks[ticker] = {
        sentiment: get_tfidf(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
        for sentiment in sentiments}

    
project_helper.print_ten_k_data([sentiment_tfidf_ten_ks[example_ticker]], sentiments)

[
  {
    negative: '[[0.         0.         0.         ... 0.        ...
    positive: '[[0.19672975 0.         0.         ... 0.        ...
    uncertainty: '[[0.         0.         0.         ... 0.00570501...
    litigious: '[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0. 0....
    constraining: '[[0.         0.         0.         ... 0.        ...
    interesting: '[[0.01889287 0.         0.         ... 0.        ...},
]


In [50]:
positive_voc = sentiment_df[sentiment_df['positive']]['word']
negative_voc = sentiment_df[sentiment_df['negative']]['word']

positive_vec = TfidfVectorizer(vocabulary=positive_voc)
positive_vec.fit(ten_ks_df['doc'].values)
negative_vec = TfidfVectorizer(vocabulary=negative_voc)
negative_vec.fit(ten_ks_df['doc'].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=9                abandon
12           abandonment
13          abandonments
51              abdicate
54            abdication
55           abdications
70              aberrant
71            aberration
72          aberrational
73           aberrations
79                  abet
138           ...  wrongful
86041         wrongfully
86049            wrongly
Name: word, Length: 1515, dtype: object)

In [47]:
def get_tfidf(vec, docs):
    
    # vec = TfidfVectorizer(vocabulary=sentiment_words)
    vec.fit(ten_ks_df['doc'].values())
    tfidf = fit_transform(docs)
    
    #return tfidf.toarray()
    return tfidf

In [61]:
pos_tfidf = positive_vec.transform(ten_ks_df['doc'])

In [67]:
test = pos_tfidf.toarray()

print(test)

[[0.18780437 0.         0.         ... 0.         0.         0.        ]
 [0.2291988  0.         0.         ... 0.         0.         0.        ]
 [0.20790583 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.49683789 0.         0.         ... 0.         0.         0.        ]
 [0.55671838 0.         0.         ... 0.         0.         0.        ]
 [0.47896312 0.         0.02605521 ... 0.         0.         0.        ]]


In [63]:
hi = pd.DataFrame(pos_tfidf.toarray())

In [64]:
hi.head()

,0,1,2,3,4,5,6,7,8,9,...,239,240,241,242,243,244,245,246,247,248
0,0.187804,0.0,0.0,0.0,0.0,0.0,0.0,0.068129,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.229199,0.0,0.0,0.0,0.0,0.0,0.0,0.066516,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.207906,0.0,0.0,0.0,0.0,0.0,0.0,0.060337,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.212847,0.0,0.0,0.0,0.0,0.0,0.0,0.043433,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.228727,0.0,0.0,0.0,0.0,0.0,0.0,0.046673,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
ten_ks_df['pos_tfidf'] = pd.Series(pos_tfidf.toarray())

Exception: Data must be 1-dimensional